# Install and Import Essential Libraries

In [ ]:
!pip install optuna
!pip install ultralytics
!pip install roboflow

In [ ]:
import optuna
import tensorflow as tf
from ultralytics import YOLO

# Getting the Dataset

I used RoboFlow

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
RF_TOKEN = user_secrets.get_secret("RF_TOKEN")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=RF_TOKEN)
project = rf.workspace("skripshit-2zmum").project("movingcamo")
version = project.version(1)
dataset = version.download("yolov12")

# Downloading the YOLOv12 Model and its decorations

In [ ]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn

In [ ]:
!wget https://github.com/sunsmarterjie/yolov12/releases/download/turbo/yolov12s.pt

In [ ]:
!pip install -U ultralytics

# Searching the best hyperparameter with Optuna

In [ ]:
import subprocess
import re
import os
import yaml
import torch
import gc
import time

def clear_gpu_memory():
    """Clear GPU memory and cache"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()
    time.sleep(2)  # Give time for cleanup

def objective(trial):
    # Clear GPU memory at the start of each trial
    clear_gpu_memory()
    
    lr0 = trial.suggest_float('lr0', 1e-5, 1e-2, log=True)
    lrf = trial.suggest_float('lrf', 1e-5, 1e-2, log=True)
    # Reduce batch size range to avoid memory issues
    batch = trial.suggest_int('batch', 8, 32)  # Reduced from 16-128
    momentum = trial.suggest_float('momentum', 0.8, 1.0)
    weight_decay = trial.suggest_float('weight_decay', 5e-4, 1e-2, log=True)
    
    # Set environment variable for memory optimization
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # Fix the command format - YOLO CLI doesn't use -- for hyperparameters
    train_command = [
        "yolo",
        "task=detect",
        "mode=train",
        f"model=/kaggle/working/yolov12s.pt",
        f"data=/kaggle/working/MovingCamo-1/data.yaml",
        f"epochs=100",  # Reduced epochs for faster trials
        f"imgsz=640",
        f"lr0={lr0}",
        f"lrf={lrf}",
        f"batch={batch}",
        f"momentum={momentum}",
        f"weight_decay={weight_decay}",
        f"project=/kaggle/working/optuna_runs",
        f"name=trial_{trial.number}",
        "verbose=False",  # Reduce output
        "patience=20",     # Early stopping
        "save=False",     # Don't save intermediate checkpoints
        "cache=False"     # Disable caching to save memory
    ]
    
    try:
        # Monitor GPU memory before training
        if torch.cuda.is_available():
            memory_before = torch.cuda.memory_allocated() / 1024**3
            print(f"Trial {trial.number} - GPU memory before: {memory_before:.2f} GB")
        
        result = subprocess.run(train_command, capture_output=True, text=True, timeout=1800)  # 30 min timeout
        
        # Debug: Print stdout and stderr to understand the output format
        print(f"Trial {trial.number} - Return code: {result.returncode}")
        if result.returncode != 0:
            print(f"STDERR: {result.stderr}")
            clear_gpu_memory()  # Clear memory even on failure
            return 0.0
            
        # Try multiple methods to extract mAP
        map_value = parse_map_from_output(result.stdout, result.stderr, trial.number)
        print(f"Trial {trial.number} - Extracted mAP: {map_value}")
        
        # Clean up trial directory to save disk space
        trial_dir = f"/kaggle/working/optuna_runs/trial_{trial.number}"
        if os.path.exists(trial_dir):
            import shutil
            try:
                # Keep only the best.pt file, remove everything else
                weights_dir = os.path.join(trial_dir, "weights")
                if os.path.exists(weights_dir):
                    for f in os.listdir(weights_dir):
                        if f != "best.pt":
                            os.remove(os.path.join(weights_dir, f))
                # Remove other directories
                for item in os.listdir(trial_dir):
                    item_path = os.path.join(trial_dir, item)
                    if os.path.isdir(item_path) and item != "weights":
                        shutil.rmtree(item_path)
            except Exception as e:
                print(f"Cleanup warning: {e}")
        
        # Clear GPU memory after each trial
        clear_gpu_memory()
        
        return map_value
        
    except subprocess.TimeoutExpired:
        print(f"Trial {trial.number} timed out")
        clear_gpu_memory()
        return 0.0
    except Exception as e:
        print(f"Trial {trial.number} failed with error: {e}")
        clear_gpu_memory()
        return 0.0

def parse_map_from_output(stdout, stderr, trial_number):
    """
    Try multiple regex patterns to find mAP value
    """
    
    # Combine stdout and stderr for searching
    full_output = stdout + "\n" + stderr
    
    # Common YOLO mAP patterns
    patterns = [
        r"mAP50-95:\s*([\d.]+)",           # mAP50-95: 0.123
        r"mAP@0\.5:0\.95:\s*([\d.]+)",     # mAP@0.5:0.95: 0.123
        r"mAP:\s*([\d.]+)",                # mAP: 0.123
        r"all\s+\d+\s+\d+\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)",  # Results table format
        r"Class\s+Images\s+Instances\s+P\s+R\s+mAP50\s+mAP50-95.*?all\s+\d+\s+\d+\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)",
        r"mAP50:\s*([\d.]+)",              # mAP50: 0.123
        r"map:\s*([\d.]+)",                # map: 0.123 (lowercase)
        r"mAP.*?:\s*([\d.]+)",             # Any mAP variant
    ]
    
    for pattern in patterns:
        match = re.search(pattern, full_output, re.IGNORECASE | re.MULTILINE)
        if match:
            # For patterns that capture multiple groups, take the last one (usually mAP50-95)
            if len(match.groups()) > 1:
                return float(match.groups()[-1])
            else:
                return float(match.group(1))
    
    # Alternative: Try to read from results file
    results_path = f"/kaggle/working/optuna_runs/trial_{trial_number}/results.csv"
    if os.path.exists(results_path):
        try:
            import pandas as pd
            df = pd.read_csv(results_path)
            if 'metrics/mAP50-95' in df.columns:
                return df['metrics/mAP50-95'].iloc[-1]  # Last epoch
            elif 'mAP_0.5:0.95' in df.columns:
                return df['mAP_0.5:0.95'].iloc[-1]
        except Exception as e:
            print(f"Error reading results file: {e}")
    
    # Try to read from last.pt weights metadata
    weights_path = f"/kaggle/working/optuna_runs/trial_{trial_number}/weights/last.pt"
    if os.path.exists(weights_path):
        try:
            import torch
            checkpoint = torch.load(weights_path, map_location='cpu')
            if 'best_fitness' in checkpoint:
                return checkpoint['best_fitness']
        except Exception as e:
            print(f"Error reading weights file: {e}")
    
    # If all methods fail, print output for debugging
    print(f"No mAP found for trial {trial_number}. Output sample:")
    print(full_output[-500:])  # Print last 500 characters
    
    return 0.0

# Alternative validation approach
def validate_and_get_map(model_path, data_path):
    """
    Run validation separately to get mAP
    """
    val_command = [
        "yolo",
        "task=detect", 
        "mode=val",
        f"model={model_path}",
        f"data={data_path}",
        "verbose=True"
    ]
    
    result = subprocess.run(val_command, capture_output=True, text=True)
    return parse_map_from_output(result.stdout, result.stderr, "validation")

# Create study with better memory management
def memory_callback(study, trial):
    """Callback to monitor and clean memory after each trial"""
    clear_gpu_memory()
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated() / 1024**3
        print(f"After trial {trial.number}: GPU memory used: {memory_used:.2f} GB")

# Set up the study with memory monitoring
study = optuna.create_study(direction='maximize')

# Test memory before starting
if torch.cuda.is_available():
    print(f"Initial GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU memory cached: {torch.cuda.memory_cached() / 1024**3:.2f} GB")

# Run fewer trials or reduce batch size if still having memory issues
print("Starting optimization with memory management...")
try:
    study.optimize(objective, n_trials=100, callbacks=[memory_callback])  # Reduced trials
    
    best_hyperparameters = study.best_params
    best_value = study.best_value
    
    print(f"Best hyperparameters: {best_hyperparameters}")
    print(f"Best mAP: {best_value}")
    
    # Save results
    import joblib
    joblib.dump(study, '/kaggle/working/optuna_study.pkl')
    
except Exception as e:
    print(f"Optimization failed: {e}")
    # Save partial results
    if len(study.trials) > 0:
        print(f"Partial results from {len(study.trials)} completed trials:")
        print(f"Best so far: {study.best_value}")
        joblib.dump(study, '/kaggle/working/optuna_study_partial.pkl')

# Final memory cleanup
clear_gpu_memory()

In [ ]:
print(best_hyperparameters)

# Training the model using the best hyperparameters from Optuna

In [ ]:
#!pip install -U ultralytics

In [ ]:
model = YOLO('yolov12s.pt')
results = model.train(
    data="/kaggle/working/MovingCamo-1/data.yaml",
    optimizer='SGD',
    epochs=100,
    patience=15,
    **best_hyperparameters
)

# Zipping the result and dowloading it

In [ ]:
!zip -r result_5.zip /kaggle/working/runs/detect/train

In [ ]:
from IPython.display import FileLink
FileLink(r'result_5.zip')